In [0]:
dbutils.secrets.list(scope = 'dbx-scope')

In [0]:
client_id = dbutils.secrets.get(scope='dbx-scope', key='dbx-app-client-id')
tenant_id = dbutils.secrets.get(scope='dbx-scope', key='dbx-app-tenant-id')
client_secret = dbutils.secrets.get(scope='dbx-scope', key='dbx-app-client-secret')
tenant_id = dbutils.secrets.get(scope='dbx-scope', key='dbx-app-tenant-id')

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

##Mounting Storage Account containers: deltahive & demo

In [0]:
dbutils.fs.mount(
  source = "abfss://deltahive@dbxdl.dfs.core.windows.net/",
  mount_point = "/mnt/deltahive",
  extra_configs = configs)

# dbutils.fs.unmount('/mnt/deltahive')

In [0]:
dbutils.fs.mount(
  source = "abfss://demo@dbxdl.dfs.core.windows.net/",
  mount_point = "/mnt/demo",
  extra_configs = configs)

# dbutils.fs.unmount('/mnt/demo')

### Show Mounts

In [0]:
display(dbutils.fs.ls('/mnt/'))

In [0]:
dbutils.fs.ls('dbfs:/mnt/deltahive/')

## Creating Delta Tables without UnityCatalgo (aka Hive Metastore)

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS MASTERCLASS_HIVEMETA
LOCATION 'dbfs:/mnt/deltahive/';

In [0]:
df = spark.read.option("inferSchema", True).csv("dbfs:/mnt/demo/circuits.csv", header=True)

In [0]:
display(df.limit(5))

In [0]:
from pyspark.sql import functions as F
cardinality_df = df.select([F.countDistinct(c).alias(c) for c in df.columns])
cardinality_df.show(truncate=False)

### file-size autotuner
It is on by default for Unity Catalog tables and for hive-metastore **tables running on DBR 11.3+**; it is bypassed only if you hard-code delta.targetFileSize or set explicit Spark overrides

Other table properties:

| Category                     | Property                              | Value                        |
|------------------------------|---------------------------------------|------------------------------|
| **Core Table Behavior**      | `delta.appendOnly`                    | `true`                       |
|                              | `delta.enableChangeDataFeed`          | `true`                       |
|                              | `delta.enableTypeWidening`            | `true`                       |
|                              | `delta.columnMapping.mode`            | `name`                       |
|                              | `delta.isolationLevel`                | `WriteSerializable`          |
| **Performance & Optimization** | `delta.autoOptimize.optimizeWrite`   | `true`                       |
|                              | `delta.autoOptimize.autoCompact`      | `true`                       |
|                              | `delta.tuneFileSizesForRewrites`      | `true`                       |
|                              | `delta.targetFileSize`                | `134217728` (128 MB in bytes)|
| **Data Skipping & Statistics** | `delta.dataSkippingNumIndexedCols`   | `32`                         |
|                              | `delta.checkpoint.writeStatsAsJson`   | `true`                       |
|                              | `delta.checkpoint.writeStatsAsStruct` | `true`                       |
| **Retention & Cleanup**      | `delta.deletedFileRetentionDuration`  | `interval 7 days`            |
|                              | `delta.logRetentionDuration`          | `interval 30 days`           |
| **Compatibility & Interoperability** | `delta.checkpointPolicy`       | `classic`                    |
|                              | `delta.enableIcebergCompatV2`         | `false`                      |
|                              | `delta.universalFormat.enabledFormats`| `iceberg`                    |
| **Advanced Features**        | `delta.enableDeletionVectors`         | `true`                       |
|                              | `delta.randomizeFilePrefixes`         | `true`                       |
|                              | `delta.randomPrefixLength`            | `8`                          |

In [0]:
%sql
CREATE TABLE IF NOT EXISTS CIRCUITS (
  circuitId        INT,
  circuitRef       STRING,
  name             STRING,
  location         STRING,
  country          STRING,
  lat              DOUBLE,
  lng              DOUBLE,
  alt              INT,
  url              STRING
)
USING DELTA
PARTITIONED BY (country)
LOCATION 'dbfs:/mnt/deltahive/circuits'
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite'   = 'true',
  'delta.autoOptimize.autoCompact'     = 'true'
);

In [0]:
df.write.format("delta").mode("overwrite").partitionBy('country').saveAsTable("MASTERCLASS_HIVEMETA.CIRCUITS")

In [0]:
%sql
SELECT COUNTRY, count(*)
FROM MASTERCLASS_HIVEMETA.CIRCUITS
GROUP BY COUNTRY
ORDER BY 2 DESC
LIMIT 5

In [0]:
%sql
DESC TABLE MASTERCLASS_HIVEMETA.CIRCUITS;

In [0]:
%sql
SHOW PARTITIONS MASTERCLASS_HIVEMETA.CIRCUITS;

In [0]:
%sql
--DROP TABLE IF EXISTS MASTERCLASS_HIVEMETA.CIRCUITS;

In [0]:
#spark.sql("DROP TABLE IF EXISTS default.circuits")